In [1]:
import pandas as pd
import requests
import warnings
import time
import multiprocessing
import threading
from concurrent.futures import ThreadPoolExecutor
# from google.colab import files
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [2]:
# uploaded = files.upload()
# df = pd.read_csv(io.BytesIO(uploaded[list(uploaded.keys())[0]]))
df = pd.read_csv('withIPA (7).csv')

In [4]:
def subset_worker_words(subset):
  print(subset)
  for row in subset.itertuples():
    print(f"Started {row.Index}")
    if pd.notna(row.ipa):
      print(f"{row.Index}:{row.ipa}")
      continue
    words=row.sentence.split(" ")
    if(len(words)>1):
      datas=""
      for word in words:
        # url= f"http://ipa.bangla.gov.bd/post_text_bangla_ipa?bangla_ipa={word}"
        url= f"http://127.0.0.1:5000//post_text_bangla_ipa?bangla_ipa={word}"
        response=None
        try:
          response = requests.get (url, verify=False)
          if response and response.status_code == 200:
            data= response.json()
            datas=datas+" "+data["data"][0]
          else:
            datas=datas+" "+"ERROR"
        except:
          print(f"error at{row.Index}:{row.sentence}")
          datas=datas+" "+"ERROR"
      print(f"Ended {row.Index}")
      df.at[row.Index,"ipa"]=datas
    else:
      url = f"http://ipa.bangla.gov.bd/post_text_bangla_ipa?bangla_ipa={row.sentence}"
      response=None
      try:
        response = requests.get (url, verify=False)
        if response and response.status_code == 200:
          data= response.json()
          df.at[row.Index,"ipa"]=data["data"][0]
          print(f"Ended {row.Index}:{row.sentence}")
          print(f"{row.Index}:{data['data'][0]}")
        else:
          data= None
      except:
        print(f"error at{row.Index}:{row.sentence}")
  # df.to_csv("withIPA.csv",index=False)
  print("Done")
  return

In [5]:
def subset_worker(subset):
  print(subset)
  for row in subset.itertuples():
    print(f"Started{row.Index}")
    if pd.notna(row.ipa):
      print(f"{row.Index}:{row.ipa}")
      continue
    url = f"http://127.0.0.1:5000//post_text_bangla_ipa?bangla_ipa={row.sentence}"
    response=None
    try:
      response = requests.get (url, verify=False)
      if response and response.status_code == 200:
        data= response.json()
        df.at[row.Index,"ipa"]=data["data"][0]
      else:
        data= None
    except:
      print(f"error at{row.Index}:{row.sentence}")
    print(f"Ended {row.Index}")
    # print(f"{row.Index}:{row.ipa}")
  # df.to_csv("withIPA.csv",index=False)
  print("Done")
  return

In [13]:
# threadpool make sure everything is currect before running this cell
start=25000
steps=50
end=26000
subsets=[]
start_pos=start
while start<end:
  subsets.append(df.iloc[start:start+steps])
  start=start+steps
print(subsets)
pool=ThreadPoolExecutor(len(subsets))
ws=[]
for subset in subsets:
  ws.append(pool.submit(subset_worker_words, subset))
print(pool.shutdown())
df.to_csv(f"withIPA{start_pos}_{end}.csv",index=False)

[       Unnamed: 0                                           sentence  ipa
25000      205844  অজ্ঞান পার্টির খপ্পড়ে পড়ে রাস্তার পাশে পড়েছিলে...  NaN
25001      134841  প্রধান শিক্ষক উপযুক্ত ব্যবস্থা নেয়ার আশ্বাস দি...  NaN
25002       94635  কাশ্মীরে সাত লাখ থেকে বাড়িয়ে ৭০ লাখ সেনা মোতায়...  NaN
25003      230550  সাধারণত এ ধরনের বেইলিব্রিজ নির্মাণে সময় দরকার ...  NaN
25004      162423  বিয়ের পর তাদের সংসারে দুই মেয়ে ও তিন ছেলে সন্ত...  NaN
25005       85846      তবে এ সমস্যার সমাধান সিভিল কোর্ট ছাড়া হবে না।  NaN
25006       44169        বরং মাঝে শাকিব অপুর প্রতি বিষোদগারও করেছেন।  NaN
25007       79362  এ ব্যাপারে বিমানবন্দর থানার ওসি গৌসুল হোসেন বল...  NaN
25008      223722  এবার দুই চিরপ্রতিদ্বন্দ্বী রিয়াল মাদ্রিদ ও বার...  NaN
25009       38925     ফলে আভ্যন্তরীণ পণ্যের বহুমুখী গমন নিশ্চিত হবে।  NaN
25010      191893          এ দুটি ছবিই সরকারি অনুদানে নির্মিত হচ্ছে।  NaN
25011      171283  ওইদিন দারুস সালাম থানার গাবতলীর বাগবড়ী শুভ্র এ...  NaN
25012      149226  ইসলামাবাদের ‘ফুল স

In [ ]:
# Fills the empty ones
subset1_1000=df.iloc[3000:6000]
for row in subset1_1000.itertuples():
  print(row.Index)
  # print(row[1])
  if pd.notna(row.ipa):
    continue
  print(row.sentence)
  data=call_api(row.sentence)
  # print(data["data"][0])
  if data:
    df.at[row.Index,"ipa"]=data["data"][0]
  else:
    print(row.Index)
    print(row.sentence)
    break
df.to_csv("withIPA.csv",index=False)

**TESTINS**

---



In [29]:
# test connection
def call_api (param):
    url = f"http://127.0.0.1:5000//transcribeBanglaTextToIpa?bangla_ipa={param}"
    response = requests.get (url, verify=False)
    if response.status_code == 200:
        return response.json()
    else:
        return None
data=call_api("আমার সোনার বাংলা।")
data["data"][0]

'amar ʃonar baŋla।'

In [ ]:
multiprocessing.cpu_count()

In [43]:
print(pool.shutdown( cancel_futures=True))
df.to_csv(f"withIPA{start_pos}_{end}.csv",index=False)

None


In [ ]:

# w0= pool.submit(subset_worker, subsets[0])
# w1=pool.submit(subset_worker, subsets[1])
# w2=pool.submit(subset_worker, subsets[2])
# w3=pool.submit(subset_worker, subsets[3])
# w4=pool.submit(subset_worker, subsets[4])
# w0.result()
# w1.result()
# w2.result()
# w3.result()
# w4.result()

# print(w0.cancel())
# print(w1.cancel())
# print(w2.cancel())
# print(w3.cancel())
# print(w4.cancel())


In [ ]:
# not working
i=500
steps=100
end=1000
subsets=[]
while i<end:
  subsets.append(df.iloc[i:i+steps])
  i=i+steps
print(subsets)

threads=[]
for subset in subsets:
  threads.append(threading.Thread(target=subset_worker, daemon=True, args=(subset,)))
for thread in threads:
  thread.start();
  time.sleep(1)
for thread in threads:
  thread.join()
print("end")